<a href="https://colab.research.google.com/github/KIMCAT33/Machine-Learning/blob/master/%EB%B6%80%EC%8A%A4%ED%8C%85_%EB%AA%A8%EB%8D%B8(XGBoost%2C_LightGBM).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<H3>Bagging(앙상블 방법) - 주어진 데이터에 대해서 여러 개의 랜덤 복원 샘플링을 하고 각 샘플의 모델링을 통해 나온 값을 평균하거나 다중 투표하는 방식으로 예측 모형의 분산을 줄여 줌으로써 예측력을 향상 시키는 방법입니다.

일반적으로 과대적합된 모형, 분산이 큰 모형에 사용하는 것이 적합합니다.

RandomForest 같은 모델입니다.



<H3>Gradient Boosting (앙상블 방법) - 회귀(Regression) 분석 or 분류 (Classificaiton)  

테이블 형태의 데이터에 대해서 엄청난 성능을 나타낸다.

LightGBM, CatBoost, XGBoost(extreme gradient boosting)이 Gradient Boosting 모델이다.
</H3>


<center>< 테이블 형태의 데이터>

![alt text](https://d13ot9o61jdzpp.cloudfront.net/images/tabular_data_1_the_perfect_format.png) </center>

부스팅 모델은 잘못 분류된 개체들에 집중하여 새로운 분류 규칙을 만드는 단계를 반복하는 방법입니다. 즉, 처음 데이터의 객체들은 동일한 가중치에서 시장하지만, 모델링을 통해 오분류된 개체들에게는 높은 가중치를 부여하고, 정분류된 객체들에게 낮은 가중치를 부여하여 오분류된 객체들이 더 잘 분류되도록 만드는 방법입니다.

![alt text](https://i.imgur.com/MvCGENh.png)

In [0]:
import pandas as pd
from sklearn.model_selection import train_test_split

# 데이터 입력
"""
 X = pd.read_csv('train.csv', index_col='Id')
 x_test_full = pd.read_csv('test.csv', index_col='Id')
"""


In [0]:
# 결측치 제거 및 레이블 분리 
"""
 X.dropna(axis=0, subset=['SalePrice'], inplace=True)
 y = X.SalePrice
 X.drop(['SalePrice'], axis=1, inplace=True)
"""


In [0]:
# 훈련 데이터와 검증 데이터 분리 

# X_train_full, X_valid_full, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=0)


In [0]:
# 카테고리 개수가 적은 columns만 뽑아보기

# low_cardinality_cols = [cname for cname in X_train_full.columns if X_train_full[cname].nunique() < 10 and X_train_full[cname].dtype == "object"]


In [0]:
# 숫자형 column만 뽑아보기

# numeric_cols = [cname for cname in X_train_full.columns if X_train_full[cname].dtype in ['int64', 'float64']]


In [0]:
# 선택된 columns으로 새로운 훈련, 검증, 테스트 데이터 셋 만들기
"""
my_cols = low_cardinality_cols + numeric_cols
X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()
X_test = x_test_full[my_cols].copy() 
"""



In [0]:
# 카테고리 타입("object") 을 숫자형(One-hot encoding)로 변환
""" 
def Categorical_to_nummeric(X_train, X_valid, X_test):
  X_train = pd.get_dummies(X_train)
  X_valid = pd.get_dummies(X_valid)
  X_test = pd.get_dummies(X_test)

  X_train, X_valid = X_train.align(X_valid, join='left', axis=1)
  X_train, X_test = X_train.align(X_test, join='left', axis=1)

  return X_train, X_valid, X_test
  """





# 랜덤 포레스트 모델 훈련시키는 함수
"""
def RandomForess_train(train_X, train_y):
  from sklearn.ensemble import RandomForestRegressor

  # Define the model. Set random_state to 1
  rf_model = RandomForestRegressor(random_state=0)

  # fit your model
  model = rf_model.fit(train_X, train_y)
  return model
"""





# XGBboost 모델 훈련시키는 함수
"""
def XGBboost_train(X_train, y_train, X_valid, y_valid):
  from xgboost import XGBRegressor
  model= XGBRegressor(n_estimator=1000, learning_rate=0.05, n_jobs=-1, random_state=0)
  model.fit(X_train, y_train,
              eval_set=[(X_valid, y_valid)],
              early_stopping_rounds=5,
              verbose=False)
  return model
"""





# 모델 성능 평가
"""
def predict_evaluate(model, X_test, y_test):
  from sklearn.metrics import mean_absolute_error
  prediction = model.predict(X_test)
  mae = mean_absolute_error(prediction, y_test)
  return mae
"""





In [0]:
 # 카테고리 데이터를 숫자형으로 인덱싱합니다. (One-Hot Encoding 방법)

X_train, X_valid, X_test = Categorical_to_nummeric(X_train, X_valid, X_test)



/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[06:26:42] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


============== XGBboost model predict score============


17748.012655179795

In [0]:
XGBboost_model = XGBboost_train(X_train, y_train, X_valid, y_valid)

print("\n\n============== XGBboost model predict score============")
predict_evaluate(XGBboost_model, X_valid, y_valid)


In [0]:
# XGBboost, LightGBM 과 Random Forest 성능 비교

import pandas as pd
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
import lightgbm as lgb

# 데이터 불러오기
"""
home_data = pd.read_csv("new_train.csv")
y = home_data.SalePrice
features = ['LotArea', 'YearBuilt', '1stFlrSF', '2ndFlrSF', 'FullBath', 'BedroomAbvGr', 'TotRmsAbvGrd']
X = home_data[features]

train_X, val_X, train_y, val_y = train_test_split(X, y, train_size=0.8, test_size=0.2,
                                                                random_state=0)
"""



/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[06:46:16] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


============== Random Forest Model predict score============
Validation MAE for Random Forest Model: 25474.17279843444




============== XGBboost model predict score============
Validation MAE for XGBboost Model: 24023.17861729452




============== LightGBM model predict score============
Validation MAE for LightGBM model: 23528.651089825627




In [0]:
#랜덤 포레스트 모델 학습
"""
rf_model = RandomForestRegressor(random_state=0)
model_rf = rf_model.fit(train_X, train_y)
prediction_rf = model_rf.predict(val_X)
mae_rf = mean_absolute_error(prediction_rf, val_y)
"""




In [0]:
#XGB Boost 모델 학습
"""
model_xgb= XGBRegressor(n_estimator=1000, learning_rate=0.05, n_jobs=-1, random_state=0)
model_xgb.fit(train_X, train_y,
            eval_set=[(val_X, val_y)],
            early_stopping_rounds=5,
            verbose=False)
prediction_xgb = model_xgb.predict(val_X)
mae_XGB = mean_absolute_error(prediction_xgb, val_y)
"""



In [0]:
#Light GBM 모델 학습

params = {'learning_rate': 0.01, 
          'max_depth': 16, 
          'boosting': 'gbdt', 
          'objective': 'regression', 
          'metric': 'auc',
          'is_training_metric': True,
          'num_leaves': 144,
          'feature_fraction': 0.9,
          'bagging_fraction': 0.7,
          'bagging_freq': 5,
          'seed':0}

          
"""
train_ds = lgb.Dataset(train_X, label = train_y)
val_ds = lgb.Dataset(val_X, label = val_y)
model_GBM = lgb.train(params,train_ds, 1000, verbose_eval=10)
prediction_light_GBM = model_GBM.predict(val_X)
mae_GBM = mean_absolute_error(prediction_light_GBM, val_y)
"""


In [0]:
# 모델 성능 비교

print("\n\n============== Random Forest Model predict score============")
print("Validation MAE for Random Forest Model: {}\n\n".format(mae_rf))

print("\n\n============== XGBboost model predict score============")
print("Validation MAE for XGBboost Model: {}\n\n".format(mae_XGB))

print("\n\n============== LightGBM model predict score============")
print("Validation MAE for LightGBM model: {}\n\n".format(mae_GBM))
